In [10]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import cv2
from keras import backend as K
import tensorflow as tf


In [19]:
# Load your model
model = load_model('my_trained_model5.h5')

# Load the image with the desired target size
img_path = 'test3.jpg'
img = image.load_img(img_path, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Make a prediction
preds = model(x, training=False)
class_idx = np.argmax(preds[0])
class_output = model.output[:, class_idx]

# Identify the name of the last convolutional layer in your model
# Adjust this according to your model architecture
last_conv_layer_name = 'conv2d_3'
last_conv_layer = model.get_layer(last_conv_layer_name)

intermediate = tf.keras.Model(model.inputs, [model.output, last_conv_layer.output])
with tf.GradientTape() as tape:
    preds, conv_outputs = intermediate(x, training=False)
    class_output = preds[:, class_idx]

# Compute gradients
grads = tape.gradient(class_output, conv_outputs)


# Average gradient per feature map
pooled_grads_value = tf.reduce_mean(grads, axis=(0, 1, 2)).numpy()

# Get the feature map for the image, already computed above
conv_layer_output_value = conv_outputs[0].numpy()

# Multiply each channel in the feature map array by the gradient importance
for i in range(conv_layer_output_value.shape[-1]):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# Obtain the heatmap
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

# Use OpenCV to overlay the heatmap on the original image
img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img

cv2.imwrite('gradcam.jpg', superimposed_img)

C:\Users\Grego\AppData\Local\Temp\ipykernel_11648\630779292.py:43: RuntimeWarning: invalid value encountered in divide
  heatmap /= np.max(heatmap)
C:\Users\Grego\AppData\Local\Temp\ipykernel_11648\630779292.py:48: RuntimeWarning: invalid value encountered in cast
  heatmap = np.uint8(255 * heatmap)


True